In [74]:
%pip install cryptography

Note: you may need to restart the kernel to use updated packages.


In [75]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.asymmetric import dh
import random



### Key exchange

In [77]:
#parameters = dh.generate_parameters(generator=2, key_size=1024, backend=default_backend())

P = 17;
G = 6;

c = random.randint(1, 1000)
public_key_c = (G**c)%P 

print(public_key_c)

with open('shared_file', 'wb') as f:
    f.write(public_key_c.to_bytes(2, "big"))
    f.close()


5
